In [1]:
# # from google.colab import auth
# # auth.authenticate_user()

# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# # 경로 변경
# cd /content/gdrive/My Drive/AIBM/20191030/

In [3]:
import pandas as pd
import numpy as np
import os

#os.chdir("C://Users//이명아//Desktop")

In [4]:
data = pd.read_csv("국가건강검진_혈압혈당데이터.csv", engine = 'python')

In [5]:
data.head()

,SEX,BTH_G,SBP,DBP,FBS,DIS,BMI
0,1,1,116,78,94,4,16.6
1,1,1,100,60,79,4,22.3
2,1,1,100,60,87,4,21.9
3,1,1,111,70,72,4,20.2
4,1,1,120,80,98,4,20.0


## 데이터 범주화 및 y라벨 설정

In [6]:
data['SBP_Y'] = data['SBP'].apply(lambda x : 0 if x < 120 else 1 ) # 위험:1 정상:0

In [7]:
data['DBP_Y'] = data['DBP'].apply(lambda x : 0 if x < 80 else 1 ) # 위험:1 정상:0

In [8]:
data['FBS_Y'] = data['FBS'].apply(lambda x : 0 if x < 126 else 1 ) # 위험:1 정상:0

In [9]:
data['BMI_Y'] = data['BMI'].apply(lambda x : 0 if x < 27 else 1 ) # 위험:1 정상:0

In [10]:
data.head()

,SEX,BTH_G,SBP,DBP,FBS,DIS,BMI,SBP_Y,DBP_Y,FBS_Y,BMI_Y
0,1,1,116,78,94,4,16.6,0,0,0,0
1,1,1,100,60,79,4,22.3,0,0,0,0
2,1,1,100,60,87,4,21.9,0,0,0,0
3,1,1,111,70,72,4,20.2,0,0,0,0
4,1,1,120,80,98,4,20.0,1,1,0,0


In [11]:
type(data)

pandas.core.frame.DataFrame

In [12]:
def healthcheck1 (row):
   if row['FBS_Y'] == 1 : #  (당뇨)
      return 1
   return 0

In [13]:
def healthcheck2(row):
    if row['FBS_Y'] + row['SBP_Y'] + row['DBP_Y'] + row['BMI_Y'] == 4 : # 당뇨 + 고혈압 + 과체중
        return 4
    if row['FBS_Y'] + row['SBP_Y'] + row['DBP_Y']  == 3 : # 당뇨 + 고혈압
        return 3
    if row['SBP_Y'] + row['DBP_Y']  == 2 : #고혈압
        return 2
    if row['FBS_Y'] == 1 :  # 당뇨
        return 1
    return 0

In [14]:
data['HEALTH1'] = data.apply(lambda row : healthcheck1(row), axis = 1)

In [15]:
data['HEALTH2'] = data.apply(lambda row : healthcheck2(row), axis = 1)

In [16]:
data.head()

,SEX,BTH_G,SBP,DBP,FBS,DIS,BMI,SBP_Y,DBP_Y,FBS_Y,BMI_Y,HEALTH1,HEALTH2
0,1,1,116,78,94,4,16.6,0,0,0,0,0,0
1,1,1,100,60,79,4,22.3,0,0,0,0,0,0
2,1,1,100,60,87,4,21.9,0,0,0,0,0,0
3,1,1,111,70,72,4,20.2,0,0,0,0,0,0
4,1,1,120,80,98,4,20.0,1,1,0,0,0,2


## 데이터 탐색

In [17]:
# 건강 체크
# 0: 정상
# 1: 당뇨
# 2: 고혈압,
# 3: 당뇨 + 고혈압
# 4: 당뇨 + 고혈압 + 과체중

data['HEALTH1'].value_counts()   #불균형 심함

0    929046
1     70954
Name: HEALTH1, dtype: int64

In [18]:
data['HEALTH2'].value_counts()   #불균형 심함

0    583320
2    345726
1     34306
3     24855
4     11793
Name: HEALTH2, dtype: int64

# 로지스틱 회귀

In [19]:
data.head()

,SEX,BTH_G,SBP,DBP,FBS,DIS,BMI,SBP_Y,DBP_Y,FBS_Y,BMI_Y,HEALTH1,HEALTH2
0,1,1,116,78,94,4,16.6,0,0,0,0,0,0
1,1,1,100,60,79,4,22.3,0,0,0,0,0,0
2,1,1,100,60,87,4,21.9,0,0,0,0,0,0
3,1,1,111,70,72,4,20.2,0,0,0,0,0,0
4,1,1,120,80,98,4,20.0,1,1,0,0,0,2


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Y = data['HEALTH1']
X = data[['SEX', 'BTH_G', 'SBP_Y', 'DBP_Y', 'BMI_Y']]

#X = data_sample.iloc[:,0:10]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3)

In [21]:
X.head()

,SEX,BTH_G,SBP_Y,DBP_Y,BMI_Y
0,1,1,0,0,0
1,1,1,0,0,0
2,1,1,0,0,0
3,1,1,0,0,0
4,1,1,1,1,0


In [22]:
log_clf = LogisticRegression()

log_clf.fit(X_train, Y_train)
log_clf.score(X_test, Y_test) 

0.9290566666666666

# 덴드로그램

In [23]:
import scipy.stats as stats
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [24]:
data_sample = data.sample(n=100, random_state = 1234)

In [25]:
columns = ['SEX','BTH_G','DIS','SBP_Y','DBP_Y','FBS_Y', 'BMI_Y', 'HEALTH1']

data_cluster= data_sample[columns]
x_columns = ['SEX','BTH_G','DIS','SBP_Y','DBP_Y','FBS_Y', 'BMI_Y']
  # 유의한 변수들만 선별(SBP 제외)
y_columns = 'HEALTH1'

In [26]:
data_cluster.head()

,SEX,BTH_G,DIS,SBP_Y,DBP_Y,FBS_Y,BMI_Y,HEALTH1
433572,1,21,4,1,0,0,0,0
254298,1,13,4,1,0,0,1,0
686334,2,12,4,0,0,0,0,0
866486,2,19,4,1,1,0,0,0
649015,2,10,4,0,0,0,0,0


In [27]:
# dendrogram 

from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Calculate the linkage: mergings
mergings = linkage(data_cluster,method='complete')

# Plot the dendrogram, using varieties as labels
plt.figure(figsize=(50,30))
dendrogram(mergings,
           labels = data_cluster.as_matrix(columns=['diabetes']),
           leaf_rotation=90,
           leaf_font_size=10,
)
plt.show()


AttributeError: 'DataFrame' object has no attribute 'as_matrix'

<Figure size 3600x2160 with 0 Axes>

# 군집분석(Kmeans) - 전체 데이터

In [ ]:
# age blood_pressure bmi fbs diastolic(이완기) dis 를 이용한 군집분석
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [ ]:
data_cluster.columns

In [ ]:
#feature = data_sample[['SEX', 'BTH_G', 'SBP', 'DBP', 'FBS', 'DIS', 'BMI', 'SBP_A', 'DBP_A','FBS_A', 'BMI_A', 'HEALTH']]
#feature = data[['SEX', 'BTH_G', 'DIS', 'SBP_Y', 'DBP_Y', 'FBS_Y', 'BMI_Y','FBS_Y']]
feature = data[['SEX', 'BTH_G', 'DIS', 'SBP_Y', 'DBP_Y', 'FBS_Y', 'BMI_Y','FBS_Y','SBP', 'DBP', 'FBS','BMI']]

In [ ]:
#label = data_sample[['diabetes']]
label = data[['HEALTH1']]

In [ ]:
feature.head()

In [ ]:
len(feature)

In [ ]:
model = KMeans(n_clusters=5,algorithm='auto')

model.fit(feature)  # 모델 학습
predict = pd.DataFrame(model.predict(feature))  # 모델 예측
predict.columns=['cluster_label']

In [ ]:
feature.reset_index(drop=True, inplace=True)
predict.reset_index(drop=True, inplace=True)

new_data = pd.concat([feature, predict], axis = 1)

In [ ]:
label.reset_index(drop = True, inplace = True)

new_data =  pd.concat([new_data, label], axis = 1)
new_data.head()

In [ ]:
new_data['cluster_label'].value_counts()

In [ ]:
#plot 시각화

centers = model.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
## 군집 별 특성 찾기 

cluster1 = new_data[new_data['cluster_label']==0] 
cluster2 = new_data[new_data['cluster_label']==1] 
cluster3 = new_data[new_data['cluster_label']==2] 
cluster4 = new_data[new_data['cluster_label']==3]  
cluster5 = new_data[new_data['cluster_label']==4]

### 클러스터별 통계량 추출

In [ ]:
def makestat(df):
    print(df.describe())
    print("***********************************************************************************")
    print(df.groupby('DIS').count())  # 성별 수 
    print("***********************************************************************************")
    print(df['HEALTH1'].value_counts()/df.count()[0]*100)    # HEALTH 구성 비율
    print("***********************************************************************************")
    print(df.groupby('SEX').count().loc[1,:][0]/df.count()[0]*100)  # 당뇨환자 비율 (에러발생 -> 당뇨환자 0인것)


In [ ]:
makestat(cluster1)

In [ ]:
makestat(cluster2)

In [ ]:
makestat(cluster3)

In [ ]:
makestat(cluster4)

In [ ]:
makestat(cluster5)

# 분산분석(ANOVA)

In [ ]:
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
cluster1.head()

In [ ]:
group1 = cluster1[['SBP', 'DBP',	'FBS',	'BMI']]
group2 = cluster2[['SBP', 'DBP',	'FBS',	'BMI']]
group3 = cluster3[['SBP', 'DBP',	'FBS',	'BMI']]
group4 = cluster4[['SBP', 'DBP',	'FBS',	'BMI']]
group5 = cluster5[['SBP', 'DBP',	'FBS',	'BMI']]

In [ ]:
group1 = group1.values.tolist()
group2 = group2.values.tolist()
group3 = group3.values.tolist()
group4 = group4.values.tolist()
group5 = group5.values.tolist()

In [ ]:
plot_data = [group1, group2, group3, group4, group5]

In [ ]:
ax = plt.boxplot(plot_data)
plt.show()

# Kmeans - 당뇨환자(diabetes==1) 군집분석

In [ ]:
data.head()

In [ ]:
data_diabetes = data[data['HEALTH1'] == 1]

In [ ]:
data_diabetes.head()

In [ ]:
data_sample = data_diabetes.sample(n=100, random_state = 1234)

In [ ]:
data_sample.head()

In [ ]:
columns = ['SEX','BTH_G','DIS','SBP_Y','DBP_Y','FBS_Y', 'BMI_Y','HEALTH2']

data_cluster= data_sample[columns]
x_columns = ['SEX','BTH_G','DIS','SBP_Y','DBP_Y','FBS_Y', 'BMI_Y',]
  # 유의한 변수들만 선별(SBP 제외)
y_columns = 'HEALTH2'

In [ ]:
# dendrogram 

from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Calculate the linkage: mergings
mergings = linkage(data_cluster,method='complete')

# Plot the dendrogram, using varieties as labels
plt.figure(figsize=(50,30))
dendrogram(mergings,
           labels = data_cluster.as_matrix(columns=['diabetes']),
           leaf_rotation=90,
           leaf_font_size=10,
)
plt.show()

In [ ]:
data_diabetes.head()

In [ ]:
feature = data[['SEX', 'BTH_G', 'DIS', 'SBP_Y', 'DBP_Y', 'FBS_Y', 'BMI_Y', 'FBS_Y','SBP', 'DBP', 'FBS','BMI']]

In [ ]:
label = data[['HEALTH2']]

In [ ]:
model = KMeans(n_clusters=4,algorithm='auto')

model.fit(feature)  # 모델 학습
predict = pd.DataFrame(model.predict(feature))  # 모델 예측
predict.columns=['cluster_label']

In [ ]:
feature.reset_index(drop=True, inplace=True)
predict.reset_index(drop=True, inplace=True)

new_data = pd.concat([feature, predict], axis = 1)

In [ ]:
label.reset_index(drop = True, inplace = True)

new_data =  pd.concat([new_data, label], axis = 1)
new_data.head()

In [ ]:
new_data['cluster_label'].value_counts()

In [ ]:
#plot 시각화

centers = model.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
## 군집 별 특성 찾기 

diabetes_cluster1 = new_data[new_data['cluster_label']==0] 
diabetes_cluster2 = new_data[new_data['cluster_label']==1] 
diabetes_cluster3 = new_data[new_data['cluster_label']==2] 
diabetes_cluster4 = new_data[new_data['cluster_label']==3]  

In [ ]:
def makestat(df):
    print(df.describe())
    print("***********************************************************************************")
    print(df.groupby('DIS').count())  # 성별 수 
    print("***********************************************************************************")
    print(df['HEALTH2'].value_counts()/df.count()[0]*100)    # HEALTH 구성 비율
    print("***********************************************************************************")
    print(df.groupby('SEX').count().loc[1,:][0]/df.count()[0]*100)  # 당뇨환자 비율 (에러발생 -> 당뇨환자 0인것)

In [ ]:
makestat(diabetes_cluster1)

In [ ]:
makestat(diabetes_cluster2)

In [ ]:
makestat(diabetes_cluster3)

In [ ]:
makestat(diabetes_cluster4)

In [ ]:
data_diabetes.head()

In [ ]:
group1_diabetes = diabetes_cluster1[['SBP', 'DBP',	'FBS',	'BMI']]
group2_diabetes = diabetes_cluster2[['SBP', 'DBP',	'FBS',	'BMI']]
group3_diabetes = diabetes_cluster3[['SBP', 'DBP',	'FBS',	'BMI']]
group4_diabetes = diabetes_cluster4[['SBP', 'DBP',	'FBS',	'BMI']]

In [ ]:
print(type(group1_diabetes), type(group2_diabetes), type(group3_diabetes), type(group4_diabetes))

In [ ]:
group1_diabetes_list = group1_diabetes.values.tolist()
group2_diabetes_list = group2_diabetes.values.tolist()
group3_diabetes_list = group3_diabetes.values.tolist()
group4_diabetes_list = group4_diabetes.values.tolist()

In [ ]:
plot_data_diabetes = [group1_diabetes_list, group2_diabetes_list, group3_diabetes_list, group4_diabetes_list]

In [ ]:
ax = plt.boxplot(plot_data_diabetes)
plt.show()

In [ ]:
data_diabetes.head()

In [ ]:
df_chi = data_diabetes

In [ ]:
contingency_table=pd.crosstab(df_chi["BTH_G"],df_chi["HEALTH1"])

In [ ]:
from statsmodels.compat.python import lrange
import numpy as np
from scipy import stats

In [ ]:
#columns = ['SEX','BTH_G','DBP','FBS','BMI','SBP_A', 'DBP_A','FBS_A', 'BMI_A','HEALTH','diabetes']

X_columns = ['SEX','BTH_G','DBP','SBP_Y', 'DBP_Y','FBS_Y', 'BMI_Y']
y_columns = ['HEALTH2']
  # 유의한 변수들만 선별(SBP 제외)
X = data_sample[X_columns]
y = data_sample[y_columns]

In [ ]:
import numpy as np
import statsmodels.api as sm

results = sm.OLS(y, X).fit()

# Inspect the results
In [16]: print(results.summary())